In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 產生二分類資料集
num_feature = 20
X, y = make_classification(n_samples=1000, n_features=num_feature, n_classes=2, random_state=42)

# 將資料分成訓練集與測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 標準化特徵
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 將資料轉換成 PyTorch tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# 定義 Logistic Regression 模型（實際上是一層神經網路）
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        # 單一線性層
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        # 前向傳播：線性變換 + Sigmoid
        return torch.sigmoid(self.linear(x))

# 初始化模型、損失函數與優化器
model = LogisticRegressionModel(input_dim=num_feature)       # Load the feature of classification task
criterion = nn.BCELoss()                                     # 使用二元交叉熵作為損失函數
optimizer = optim.SGD(model.parameters(), lr=0.01)           # 使用隨機梯度下降法

# 訓練模型
num_epochs = 1000
for epoch in range(num_epochs):
    # 前向傳播
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # 反向傳播和優化
    optimizer.zero_grad()  # 梯度歸零
    loss.backward()  # 反向傳播
    optimizer.step()  # 更新權重
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 測試模型
with torch.no_grad():  # 測試時不需要計算梯度
    predicted = model(X_test)
    predicted = (predicted > 0.5).float()  # 閾值設為 0.5 進行分類
    accuracy = (predicted == y_test).sum() / y_test.size(0)
    print(f'Accuracy: {accuracy:.4f}')


Epoch [100/1000], Loss: 0.5407
Epoch [200/1000], Loss: 0.4644
Epoch [300/1000], Loss: 0.4243
Epoch [400/1000], Loss: 0.3996
Epoch [500/1000], Loss: 0.3829
Epoch [600/1000], Loss: 0.3709
Epoch [700/1000], Loss: 0.3620
Epoch [800/1000], Loss: 0.3550
Epoch [900/1000], Loss: 0.3495
Epoch [1000/1000], Loss: 0.3451
Accuracy: 0.8400
